# Enhanced UNet++ with CBAM and ASPP
This notebook contains the implementation of Enhanced UNet++ with CBAM attention and ASPP for retinal vessel segmentation.

In [ ]:
from tensorflow.keras.layers import (Input, Conv2D, BatchNormalization, Activation, 
                                     UpSampling2D, Concatenate, GlobalAveragePooling2D, 
                                     Reshape, Dropout, Multiply, Add, MaxPooling2D, DepthwiseConv2D)
from tensorflow.keras.models import Model

def atrous_spatial_pyramid_pooling(x):
    pool = GlobalAveragePooling2D()(x)
    pool = Reshape((1, 1, x.shape[-1]))(pool)
    pool = Conv2D(x.shape[-1], kernel_size=1)(pool)
    pool = UpSampling2D(size=(x.shape[1], x.shape[2]), interpolation='bilinear')(pool)
    conv1 = Conv2D(x.shape[-1], kernel_size=1, dilation_rate=1, padding='same')(x)
    conv6 = Conv2D(x.shape[-1], kernel_size=3, dilation_rate=6, padding='same')(x)
    conv12 = Conv2D(x.shape[-1], kernel_size=3, dilation_rate=12, padding='same')(x)
    conv18 = Conv2D(x.shape[-1], kernel_size=3, dilation_rate=18, padding='same')(x)
    combined = Concatenate()([pool, conv1, conv6, conv12, conv18])
    return Conv2D(x.shape[-1], kernel_size=1)(combined)

def cbam_block(input_feature, ratio=8):
    channel = input_feature.shape[-1]
    avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = Dense(channel // ratio, activation='relu')(avg_pool)
    avg_pool = Dense(channel, activation='sigmoid')(avg_pool)
    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Dense(channel // ratio, activation='relu')(max_pool)
    max_pool = Dense(channel, activation='sigmoid')(max_pool)
    channel_attention = Add()([avg_pool, max_pool])
    channel_attention = Multiply()([input_feature, channel_attention])
    avg_pool = Lambda(lambda x: K.mean(x, axis=-1, keepdims=True))(channel_attention)
    max_pool = Lambda(lambda x: K.max(x, axis=-1, keepdims=True))(channel_attention)
    spatial_attention = Concatenate(axis=-1)([avg_pool, max_pool])
    spatial_attention = Conv2D(1, kernel_size=7, activation='sigmoid', padding='same')(spatial_attention)
    return Multiply()([channel_attention, spatial_attention])

def enhanced_unetplusplus(input_shape):
    inputs = Input(input_shape)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2, 2))(conv1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2, 2))(conv2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2, 2))(conv3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D((2, 2))(conv4)
    bottleneck = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool4)
    bottleneck = atrous_spatial_pyramid_pooling(bottleneck)
    up4 = UpSampling2D((2, 2))(bottleneck)
    att4 = cbam_block(conv4)
    dec4 = Conv2D(512, (3, 3), activation='relu', padding='same')(Concatenate()([up4, att4]))
    up3 = UpSampling2D((2, 2))(dec4)
    att3 = cbam_block(conv3)
    dec3 = Conv2D(256, (3, 3), activation='relu', padding='same')(Concatenate()([up3, att3]))
    up2 = UpSampling2D((2, 2))(dec3)
    att2 = cbam_block(conv2)
    dec2 = Conv2D(128, (3, 3), activation='relu', padding='same')(Concatenate()([up2, att2]))
    up1 = UpSampling2D((2, 2))(dec2)
    att1 = cbam_block(conv1)
    dec1 = Conv2D(64, (3, 3), activation='relu', padding='same')(Concatenate()([up1, att1]))
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(dec1)
    return Model(inputs, outputs)